# Deep Neural Network

## Preliminar Settings

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Import the data

In [3]:
labels = pd.read_csv('marksheet.csv')
dataset = pd.read_csv('PI-CAI_features')

# Filter the dataset to include only AI annotations and t2w sequences and ROI lesion
filtered_dataset = dataset[dataset['annotator'] == 'AI']
filtered_dataset = filtered_dataset[filtered_dataset['sequence'] == 't2w']
filtered_dataset = filtered_dataset[filtered_dataset['ROI'] == 'lesion']
filtered_dataset.drop(columns=['annotator', 'sequence', 'ROI_ID', 'ROI','img_path', 'seg_path', 'extraction_ID'], inplace=True)
dataset = filtered_dataset

# Rename columns to match the labels dataset
labels.rename(columns={'patient_id': 'patient_ID', 'study_id': 'study_ID'}, inplace=True)

# We only want Magnetic Resonace Biopsy (MRBx) labels because Systematic Biopsy (SBx) labels are not for our usecase
# we also remove those that have both because clicinians result might be biased
#labels = labels[labels['histopath_type'] == 'MRBx']

# Remove useless columns from labels dataset
labels.drop(columns=['mri_date', 'histopath_type', 'center', 'lesion_ISUP', 'lesion_GS'], inplace=True)

# Rename Yes to 1 and No to 0 in the labels dataset
labels.case_csPCa = labels.case_csPCa.map(lambda x: 1 if x == 'YES' else 0)

# Drop Missing values
labels.dropna(inplace=True)

# Remove High Correlated Features
corr_matrix = dataset.drop(columns=['study_ID', 'patient_ID']).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
dataset.drop(to_drop, axis=1, inplace=True)

# Merge data
merge = pd.merge(dataset, labels, on=['patient_ID', 'study_ID'], how='inner')
merge.drop(columns=['patient_ID', 'study_ID'], inplace=True)


isup = 'case_ISUP'
pca = 'case_csPCa'

# Study case: IS ISUP>=3
merge['ISUP>=3'] = merge[isup].map(lambda x: 1 if x >= 3 else 0)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(merge.drop(columns=[isup, pca, 'ISUP>=3']), merge['ISUP>=3'], test_size=0.2, random_state=42)

## Convert into Pytorch supported

In [4]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
train_ds = TabularDataset(X_train.values, y_train.values)
test_ds = TabularDataset(X_test.values, y_test.values)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

## Create the Network

In [159]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(317, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## Train And Eval

In [160]:
def train(model, criterion, optimizer, epoch_count=10, verbose=True):
    print(f'Training')
    model = model.to(device)
    for epoch in range(epoch_count):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze()
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if verbose:
            print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
    return

def eval(model):
    print(f'Eval')
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = torch.sigmoid(model(xb).squeeze()) > 0.5
            correct += (pred == yb).sum().item()
            total += yb.size(0)
        print(f"Accuracy: {correct / total:.2%}")
    return

def train_and_eval(model, criterion, optimizer, epoch_count=10, verbose=True):
    train(model=model, criterion=criterion, optimizer=optimizer, epoch_count=epoch_count, verbose=verbose)
    eval(model)
    return

## CODES

In [162]:
model = NeuralNetwork()

lr = 0.0001
epochs = 200

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_and_eval(model=model, criterion=criterion, optimizer=optimizer, epoch_count=epochs, verbose=False)

Training
Eval
Accuracy: 54.39%
